In [1]:
filename = "data/peak_areas_yw_hap1_wt.csv" #path to datafile

numCarbons = 16
name = "16:0 Palmitic acid"
NAAname = "NAA"
aspName = "Asp"

In [2]:
#import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import os
matplotlib.rcParams['figure.dpi'] = 100
import helper
import molmass
from PIL import Image
from multiprocessing import Pool
import importlib
importlib.reload(helper)
import zipfile
from sklearn.metrics import r2_score
import seaborn as sb

def getISAEq(numCarbons):
    d = {16:helper.palmitateISA,14:helper.myristicISA,18:helper.stearicISA,20:helper.arachidonicISA,22:helper.dhaISA}
    return d[numCarbons]

In [3]:
func = getISAEq(numCarbons) #get relevant ISA equation

#load in data and organize
data = pd.read_csv(filename,index_col=[0,1])
samples = data.columns.values[1:]
print(samples)
data

['WT1_60H_84 Area' 'WT2_60H_90 Area' 'WT3_60H_92 Area']


Precursor Mz  WT1_60H_84 Area  WT2_60H_90 Area  \
Molecule Name      label                                                   
Asp                0        132.030232           260570           343018   
                   1        133.033587            28688            46401   
                   2        134.036941           121703           155371   
                   3        135.040296            74015           101356   
                   4        136.043651            36610            43449   
NAA                0        174.040796          2951969          2773518   
                   1        175.044151           611701           496967   
                   2        176.047506         11352694         10558179   
                   3        177.050861          1851031          1733510   
                   4        178.054216          4286317          3870382   
                   5        179.057571          2186001          1965152   
                   6        180.060925           835492           703768   
16:0 Palmitic acid 0        255.232954          2526652          2123345   
                   1        265.266503           252369           189343   
                   2        266.269858            70498            49567   
                   3        267.273212           407744           237607   
                   4        268.276567            74370            50896   
                   5        269.279922           371036           286929   
                   6        270.283277            76282            42543   
                   7        271.286632           301722           128504   
                   8        256.236309           419743           330058   
                   9        257.239664            64160            47227   
                   10       258.243019             7142             5916   
                   11       259.246374            18347            13845   
                   12       260.249729             5740             5815   
                   13       261.253083            47955            35683   
                   14       262.256438            22478            13311   
                   15       263.259793           118733            89806   
                   16       264.263148            42793            28343   

                          WT3_60H_92 Area  
Molecule Name      label                   
Asp                0               584643  
                   1                64820  
                   2               252762  
                   3               154391  
                   4                59645  
NAA                0              2731776  
                   1               549869  
                   2             10220170  
                   3              1602416  
                   4              3658590  
                   5              1945442  
                   6               699502  
16:0 Palmitic acid 0              1841743  
                   1               273001  
                   2                68376  
                   3               340531  
                   4                66392  
                   5               384487  
                   6                50273  
                   7               145202  
                   8               313524  
                   9                43336  
                   10                6971  
                   11               21173  
                   12               10984  
                   13               64097  
                   14               22586  
                   15              153129  
                   16               45175

In [4]:
#gather images for mzs of interest 
palmitateInds = []
for i in range(numCarbons + 1):
    for ind in data.index.values:
        if ind[0] == name and ind[1] == i:
            palmitateInds.append(ind)

productDf = data.loc[palmitateInds,samples]
for samp in samples:
    productDf[samp] = productDf[samp]/productDf[samp].sum()
productDf

WT1_60H_84 Area  WT2_60H_90 Area  WT3_60H_92 Area
Molecule Name      label                                                   
16:0 Palmitic acid 0             0.523359         0.577194         0.478253
                   1             0.052275         0.051470         0.070891
                   2             0.014603         0.013474         0.017755
                   3             0.084458         0.064589         0.088427
                   4             0.015405         0.013835         0.017240
                   5             0.076855         0.077997         0.099841
                   6             0.015801         0.011565         0.013055
                   7             0.062497         0.034932         0.037705
                   8             0.086944         0.089720         0.081414
                   9             0.013290         0.012838         0.011253
                   10            0.001479         0.001608         0.001810
                   11            0.003800         0.003764         0.005498
                   12            0.001189         0.001581         0.002852
                   13            0.009933         0.009700         0.016644
                   14            0.004656         0.003618         0.005865
                   15            0.024594         0.024412         0.039764
                   16            0.008864         0.007705         0.011731

In [6]:
c13ab = 0.011 #natural abundance
N = [(1-c13ab)**2,2*(1-c13ab)*c13ab,c13ab**2] #get expected labeling of precursor from natural abundance
T = [0,0,1] #set initial tracer labeling
outputDf = {}
for samp in samples:
    g,D,T_found,err,P_pred = helper.ISAFit(T,N,productDf[samp].values,func,list(range(len(productDf[samp].values))),.5)
    outputDf[samp] = {"g(t)":g,"X0":T_found[0],"X1":T_found[1],"X2":T_found[2],"SSE":err}
    for x in range(len(P_pred)):
        outputDf[samp]["P"+str(x)] = P_pred[x]
outputDf = pd.DataFrame.from_dict(outputDf,orient="index").transpose()
outputDf
outputDf.to_csv(filename.replace(".csv","_ISA_result"))

In [7]:
#aspInds = []
#for i in range(4 + 1):
#    for ind in data.index.values:
#        if ind[0] == aspName and ind[1] == i:
#            aspInds.append(ind)
#
#aspDf = data.loc[aspInds,samples]
#for samp in samples:
#    aspDf[samp] = aspDf[samp]/aspDf[samp].sum()
#aspDf

In [8]:
#naaInds = []
#for i in range(6 + 1):
#    for ind in data.index.values:
#        if ind[0] == NAAname and ind[1] == i:
#            naaInds.append(ind)
#
#naaDf = data.loc[naaInds,samples]
#for samp in samples:
#    naaDf[samp] = naaDf[samp]/naaDf[samp].sum()
#naaDf

In [9]:
#outputDf2 = {}
#for samp in samples:
#    T,err,fit = helper.findAcetylCoa(naaDf[samp].values,aspDf[samp].values)
#    outputDf2[samp] = {"X0_2":T[0],"X1_2":T[1],"X2_2":T[2],"SSE_2":err}
#    for x in range(len(fit)):
#        outputDf2[samp]["NAA_fit M" + str(x)] = fit[x]
#
#outputDf2 = pd.DataFrame.from_dict(outputDf2,orient="index").transpose()
#outputDf2

In [10]:
#totalRes = pd.concat((outputDf,outputDf2),axis=0,ignore_index=False)
#totalRes.to_csv(filename.replace(".csv","_ISA_results.csv"))
#totalRes

In [11]:
#goodSamps = [x for x in samples if not np.isnan(outputDf2.at["SSE_2",x]) and outputDf2.at["SSE_2",x] < 0.05 ]
# print(goodSamps)
# for x in range(7):
#     plt.figure()
#     plt.scatter(naaDf.loc[("NAA",x),goodSamps].values,outputDf2.loc["NAA_fit M" + str(x),goodSamps].values)
#     maxx = np.max(naaDf.loc[("NAA",x),goodSamps].values)
#     maxy = np.max(outputDf2.loc["NAA_fit M" + str(x),goodSamps].values)
#     plt.plot([0,np.max([maxx,maxy])],[0,np.max([maxx,maxy])])
#     plt.title(x)
#     plt.xlabel("meaused NAA")
#     plt.ylabel("fit NAA")
#     plt.title("NAA M" + str(x))
    
# plt.figure()

#x_ind = 0
#x_lab = []
#for x in range(7):
#    m1 = np.mean(naaDf.loc[("NAA",x),goodSamps].values)
#    s1 = np.std(naaDf.loc[("NAA",x),goodSamps].values)
#    m2 = np.mean(outputDf2.loc["NAA_fit M" + str(x),goodSamps].values)
#    s2 = np.std(outputDf2.loc["NAA_fit M" + str(x),goodSamps].values)
#    plt.bar([x_ind,x_ind+1],[m1,m2],color=["red","black"],yerr = [s1,s2],capsize=3)
#    x_lab.append([x_ind+.5,"M+" + str(x)])
#    x_ind += 4
#plt.title("red = measured NAA, black = fit NAA")
#plt.xticks([x[0] for x in x_lab],[x[1] for x in x_lab],rotation=90)

In [12]:
#plt.figure()
#
#x_ind = 0
#x_lab = []
#for x in range(3):
#    m1 = np.mean(totalRes.loc["X"+str(x),goodSamps].values)
#    s1 = np.std(totalRes.loc["X"+str(x),goodSamps].values)
#    m2 = np.mean(outputDf2.loc["X" + str(x) + "_2",goodSamps].values)
#    s2 = np.std(outputDf2.loc["X" + str(x) + "_2",goodSamps].values)
#    plt.bar([x_ind,x_ind+1],[m1,m2],color=["green","blue"],yerr = [s1,s2],capsize=3)
#    x_lab.append([x_ind+.5,"M+" + str(x)])
#    x_ind += 4
#plt.xticks([x[0] for x in x_lab],[x[1] for x in x_lab],rotation=90)
#plt.title("green = ISA inferred, blue = NAA&Asp inferred")


In [13]:
#plt.figure()
#
#x_ind = 0
#x_lab = []
#for x in range(3):
    #m1 = np.mean(totalRes.loc["X"+str(x),goodSamps].values)
    #s1 = np.std(totalRes.loc["X"+str(x),goodSamps].values)
#    m2 = np.mean(outputDf2.loc["X" + str(x) + "_2",goodSamps].values)
#   s2 = np.std(outputDf2.loc["X" + str(x) + "_2",goodSamps].values)
#    plt.bar([x_ind],[m2],color=["blue"],yerr = [s2],capsize=3)
#    x_lab.append([x_ind+.5,"M+" + str(x)])
#    x_ind += 2
#plt.xticks([x[0] for x in x_lab],[x[1] for x in x_lab],rotation=90)
#plt.title("blue = NAA&Asp inferred")
